# EDA

In [21]:
import numpy as np
import pandas as pd

import os
import glob, datetime

In [11]:
BASE_DIR = os.path.join(os.getcwd(), os.pardir)
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')

TYPE_LABEL = {'clicks':0, 'carts':1, 'orders':2}

## ファイルimport

In [12]:
def read_file_to_cache(f):
    df = pd.read_parquet(f)
    df.ts = (df.ts/1000).astype('int32')
    df['type'] = df['type'].map(TYPE_LABEL).astype('int8')
    return df

In [74]:
%%time
# # ================
# # sample_submission
# # ================
path_sample_submission = os.path.join(BASE_DIR, 'data', 'original_data', 'sample_submission.csv')
sample_sub = pd.read_csv(path_sample_submission)


# # ================
# # train/test parquet
# # ================
# train_paths = glob.glob(os.path.join(DATA_DIR, 'chunked_data', 'train_parquet', '*'))
# test_paths = glob.glob(os.path.join(DATA_DIR, 'chunked_data', 'test_parquet', '*'))
# # data_cache_train = {}
# # data_cache_test = {}
# # for p in train_paths: data_cache_train[p] = read_file_to_cache(p)
# # for p in test_paths: data_cache_test[p] = read_file_to_cache(p)

# ================
# validation file
# ================
train_val_paths = glob.glob(os.path.join(DATA_DIR, 'validation', 'train_parquet', '*'))
test_val_paths = glob.glob(os.path.join(DATA_DIR, 'validation', 'test_parquet', '*'))
test_label = os.path.join(DATA_DIR, 'validation', 'test_labels.parquet')
validation_train = {}
validation_test = {}
for p in train_val_paths: validation_train[p] = read_file_to_cache(p)
for p in test_val_paths: validation_test[p] = read_file_to_cache(p)
validation_label = pd.read_parquet(test_label)

CPU times: total: 3.39 s
Wall time: 3.41 s


In [81]:
# train/test
display( list( data_cache_train.values() )[0].sample() )
display( list( data_cache_test.values() )[0].sample() )

# validation
display( list( validation_train.values() )[0].sample() )
display( list( validation_test.values() )[0].sample() )

# sample submission


,session,aid,ts,type
2442641,46554,562295,1661344534,0


,session,aid,ts,type
137246,12926064,442580,1661760349,0


,session,aid,ts,type
732402,14839,585934,1660418590,0


,session,aid,ts,type
181091,11132579,1059808,1661151349,0


## EDA

- データ期間
    - configuration
        - train: 2022/8/1 - 2022/8/29(28 days)
        - test : 2022/8/29 - 2022/9/5(7 days)
    - validation
        - train: 2022/8/1 - 2022/8/22(21 days)
        - test : 2022/8/22 - 2022/8/29(7 days)
        - validation: 2022/8/29 - 2022/9/5(7 days)

In [39]:
def _get_ts_range(data_d):
    min_list = []
    max_list = []
    for k,v in data_d.items():
        min_ts = v.ts.min()
        max_ts = v.ts.max()
        min_list.append( datetime.datetime.fromtimestamp(min_ts) )
        max_list.append( datetime.datetime.fromtimestamp(max_ts) )

    min_ = min(min_list)
    max_ = min(max_list)

    print(f'min date: {min_}')
    print(f'max date: {max_}')

# configuration
_get_ts_range(data_cache_train)
_get_ts_range(data_cache_test)

# validation
_get_ts_range(validation_train)
_get_ts_range(validation_test)


min date: 2022-08-01 07:00:00
max date: 2022-08-29 06:59:43
min date: 2022-08-29 07:00:00
max date: 2022-09-05 06:57:21
min date: 2022-08-01 07:00:00
max date: 2022-08-22 06:59:50
min date: 2022-08-22 07:00:00
max date: 2022-08-29 06:52:30


- validationデータの切り方
    - validateion_test: term2(8/22 - 29), validation(8/29 - 9/5)ともにactivityあるsessionの内、term2内のactivity
    - validation_label: term2(8/22 - 29), validation(8/29 - 9/5)ともにactivityあるsessionの内、validation内のactivity
        - validation_test, validation_labelのsessionは完全一致

In [65]:
# 8/29 - 9/5期間内の全session
sessions_in_term3 = []
for k,v in data_cache_test.items():
    print('=========')
    print(os.path.basename(k))
    sessions = v['session'].unique().tolist()
    print(min(sessions))
    print(max(sessions))
    sessions_in_term3.extend(sessions)

# 8/22 - 8/29期間内の全session
sessions_in_term2 = []
for k,v in validation_test.items():
    print('=========')
    print(os.path.basename(k))
    sessions = v['session'].unique().tolist()
    print(min(sessions))
    print(max(sessions))
    sessions_in_term2.extend(sessions)

# validation_label
sessions_valid = validation_label.session.unique().tolist()


000000000_000100000.parquet
12899779
12999778
000100000_000200000.parquet
12999779
13099778
000200000_000300000.parquet
13099779
13199778
000300000_000400000.parquet
13199779
13299778
000400000_000500000.parquet
13299779
13399778
000500000_000600000.parquet
13399779
13499778
000600000_000700000.parquet
13499779
13599778
000700000_000800000.parquet
13599779
13699778
000800000_000900000.parquet
13699779
13799778
000900000_001000000.parquet
13799779
13899778
001000000_001100000.parquet
13899779
13999778
001100000_001200000.parquet
13999779
14099778
001200000_001300000.parquet
14099779
14199778
001300000_001400000.parquet
14199779
14299778
001400000_001500000.parquet
14299779
14399778
001500000_001600000.parquet
14399779
14499778
001600000_001700000.parquet
14499779
14571581
000.parquet
11098528
11188590
001.parquet
11188591
11278653
002.parquet
11278654
11368716
003.parquet
11368717
11458779
004.parquet
11458780
11548842
005.parquet
11548843
11638905
006.parquet
11638906
11728968
007.parq

In [77]:
print(len(sessions_in_term2))
print(len(sessions_in_term3))
print(len(sessions_valid))

# term2とvalidのsessionは完全一致
term2_x_valid = list( set(sessions_in_term2) & set(sessions_valid) )
print(len(term2_x_valid))

# term2とterm3のsessionは一致しない
term2_x_term3 = list( set(sessions_in_term2) & set(sessions_in_term3) )
print(len(term2_x_term3))

# term2とterm3のsessionは一致しない
term2_x_term3 = list( set(sessions_in_term2) & set(sessions_in_term3) )
print(len(term2_x_term3))

1801251
1671803
1801251
1801251
0


In [15]:
# 1sessionあたりのaid(商品), ts(時刻)のcount, nunique(重複)確認
def _check_ct_duplicate(i):
    print(f'file list index: {i}')
    k = train_val_paths[i]
    g = validation_train[k].groupby('session').agg({'aid':['count', 'nunique'], 'ts': ['count', 'nunique']})
    g.columns = ['_'.join(c) for c in g.columns]
    display(g.describe())

_check_ct_duplicate(0)
_check_ct_duplicate(-1)

file list index: 0


,aid_count,aid_nunique,ts_count,ts_nunique
count,110986.000000,110986.000000,110986.000000,110986.000000
mean,42.921783,24.946777,42.921783,42.470113
std,63.751143,35.157395,63.751143,63.040910
min,1.000000,1.000000,1.000000,1.000000
25%,5.000000,4.000000,5.000000,5.000000
50%,16.000000,10.000000,16.000000,16.000000
75%,52.000000,31.000000,52.000000,51.000000
max,495.000000,460.000000,495.000000,495.000000


file list index: -1


,aid_count,aid_nunique,ts_count,ts_nunique
count,110914.000000,110914.000000,110914.000000,110914.000000
mean,5.918027,4.024163,5.918027,5.826776
std,8.903743,5.473617,8.903743,8.635322
min,1.000000,1.000000,1.000000,1.000000
25%,2.000000,1.000000,2.000000,2.000000
50%,3.000000,2.000000,3.000000,3.000000
75%,6.000000,4.000000,6.000000,6.000000
max,246.000000,168.000000,246.000000,245.000000


# Co-Visitation Matrix

In [5]:
s = 'setting0'
path = os.path.join(OUTPUT_DIR, 'covisit', s, '*.pqt')
files = glob.glob(path)

In [6]:
top = 20
adds = []
for f in files:
    add = pd.read_parquet(files[0])
    add = add[add.n < top].copy()
    adds.append(add)